In [12]:
import pandas as pd
import os
from glob import glob

#### 특정 경로에 있는 데이터 파일을 모두 하나로 결합하는 함수
1. 특정 경로에 있는 파일의 목록을 로드
2. 파일의 목록에서 특정 확장자를 필터링
3. 필터링된 목록을 기준으로 반복문을 실행
    - 해당하는 목록의 파일 하나씩을 데이터프레임으로 로드
    - 비어있는 데이터프레임 안에 단순한 행 결합
    - 만약에 로드 중 인코딩 에러 발생 시 다른 engine을 이용
4. 결합된 데이터프레임을 되돌려 준다.

In [14]:
def read_df (_path, _encoding="utf-8"):
    # 경로와 파일을 입력하면 확장자를 분리한다.
    head, tail = os.path.splitext(_path)
    # head: csv/2017/201701_expense_list
    # tail: .csv

    if tail == ".csv":
        df=pd.read_csv(_path, encoding=_encoding)
    elif tail == ".json":
        df=pd.read_json(_path, encoding=_encoding)
    elif tail == ".xml":
        df=pd.read_xml(_path, encoding=_encoding)
    elif tail in [".xlsx", ".xls"]:
        df=pd.read_excel(_path, engine=_encoding)
    else:
        df=pd.DataFrame()

    return df

In [ ]:
read_df("csv/2017/201701_expense_list.csv")

In [16]:
def data_load_add(_path, _end="csv"):
    #_path: 파일의 경로
    #_end: 파일의 확장자

    # 파일의 경로를 기준으로 파일의 목록을 로드하여 변수에 저장
    file_list=os.listdir(_path)
    _path += "/"

    # 파일의 목록에서 _end의 확장자를 가진 파일의 목록을 제외하고 모두 삭제(필터)
    # case1: '.'을 기준으로 파일의 이름을 나눈다 -> 마지막 값 확인
        # print(file_list[0].split(".")[-1])
    # case2: 문자의 마지막이 csv라면
        # print(file_list[0].endswith(_end))
    # case3: os 라이브러리를 이용하여 파일의 이름과 확장자를 나눈다
        # print(os.path.splitext(file_list[0]))

    # case4: for문 이용 - 한줄 작성법, 새로운 리스트 생성
        # i for i in range(9) if i %2==0 ---> [0,2,4,6,8]
    
    list_filter=[
        i for i in file_list if i.endswith(_end) 
                            # if i.split(".")[-1]==_end
    ]
    result=pd.DataFrame()
    # 반복문을 이용하여 데이터를 로드하고 비어있는 데이터프레임에 단순한 행 결합
    for i in list_filter:
        try:
            df=read_df(_path+i)
        except: # read_df 호출 시 에러가 발생한다면
            try:
                df=read_df(_path+i, "CP949")
            except:
                df=read_df(_path+i, "EUC-KR")
        result=pd.concat([result, df])
    return result


In [ ]:
data_load_add("csv/2021")

In [19]:
# 데이터프레임을 파일로 저장
# 데이터프레임이라는 타입을 csv, xlsx, json 타입으로 변경
# to 키워드 + 타입
# csv 저장: to_csv, 엑셀 저장: to_excel
# indax 매개변수: 인덱스를 파일에 저장할 것인가?
df_2021=data_load_add("csv/2021")
df_2021.to_csv("df_2021_add.csv", index=False) # 현재 작업중인 경로에 csv 파일로 저장됨

In [ ]:
pd.read_csv("df_2021_add.csv")

#### 특정경로에 있는 파일들을 데이터프레임으로 로드하면서 각각 다른 변수에 저장하는 함수
1. 특정 경로에 있는 파일의 목록을 로드
2. 목록을 기준으로 반복문 실행
    - 경로, 파일을 나눈다.
    - 파일을 이름과 확장자로 나눈다.
    - 앞서 생성한 read_df 함수를 이용하여 데이터프레임 로드
    - 전역 변수의 이름으로 파일의 이름을 지정하여 데이터프레임 대입

In [24]:
for i in range(9):
    a=i

print(a) # 들여쓰기 여부에 따라 출력값이 다름

8


In [ ]:
# 전역변수가 저장되어 있는 공간을 출력
globals()

In [26]:
globals()["b"]="test"

In [27]:
b

'test'

In [28]:
def data_load(_path):
    # 입력받은 경로를 기준으로 파일의 목록을 로드
    # glob
    file_list=glob(_path+"/*.*")

    for i in file_list:
        # 파일리스트의 경로와 파일 나눈다.
        forder, name = os.path.split(i)
        # name의 파일명과 확장자를 나눈다.
        head, tail = os.path.splitext(name)
        print(head, "\t", tail) # "\t" 확인용 띄어쓰기

        try:
            df=read_df(i)
        except:
            try:
                df=read_df(i, "CP949")
            except:
                df=read_df(i, "EUC-KR")
        # 전역변수에 head 값을 사용한다.
        globals()[head]= df.copy()

In [29]:
data_load("csv/2017")

201701_expense_list 	 .csv
201702_expense_list 	 .csv
201703_expense_list 	 .csv
201704_expense_list 	 .csv
201705_expense_list 	 .csv
201706_expense_list 	 .csv
201707_expense_list 	 .csv
201708_expense_list 	 .csv
201709_expense_list 	 .csv
201710_expense_list 	 .csv
201711_expense_list 	 .csv
201712_expense_list 	 .csv


In [30]:
globals()["201701_expense_list"]

,nid,title,url,dept_nm_lvl_1,dept_nm_lvl_2,dept_nm_lvl_3,dept_nm_lvl_4,dept_nm_lvl_5,exec_yr,exec_month,expense_budget,expense_execution,category,dept_nm_full,exec_dt,exec_loc,exec_purpose,target_nm,payment_method,exec_amount
0,11430252,2017년 1월 장애인복지정책과 업무추진비 집행내역,http://opengov.seoul.go.kr/public/11430252,서울시본청,복지본부,장애인복지정책과,NaN,NaN,2017,1,NaN,NaN,NaN,복지본부 장애인복지정책과,2017-01-26 13:10,동해일식 (중구 무교동),기본소득과 장애인복지 논의간담회,장애인복지정책팀장 외 2명,카드,76000
1,11430252,2017년 1월 장애인복지정책과 업무추진비 집행내역,http://opengov.seoul.go.kr/public/11430252,서울시본청,복지본부,장애인복지정책과,NaN,NaN,2017,1,NaN,NaN,NaN,복지본부 장애인복지정책과,2017-01-25 22:41,김앤장 (중구 무교로),장애인단체 활동지원 논의간담회,장애인복지정책과장 외 3명,카드,102000
2,11430252,2017년 1월 장애인복지정책과 업무추진비 집행내역,http://opengov.seoul.go.kr/public/11430252,서울시본청,복지본부,장애인복지정책과,NaN,NaN,2017,1,NaN,NaN,NaN,복지본부 장애인복지정책과,2017-01-24 12:35,왕왕생고기 (중구 을지로),장애인 기본돌봄 복지시책 논의간담회,장애인복지정책팀장외7명,카드,80000
3,11430252,2017년 1월 장애인복지정책과 업무추진비 집행내역,http://opengov.seoul.go.kr/public/11430252,서울시본청,복지본부,장애인복지정책과,NaN,NaN,2017,1,NaN,NaN,NaN,복지본부 장애인복지정책과,2017-01-24 12:23,서울불고기 (중구 남대문로),서울시일자리통합지원센터 운영개선 논의간담회,장애인복지정책과장외5명,카드,112000
4,11430252,2017년 1월 장애인복지정책과 업무추진비 집행내역,http://opengov.seoul.go.kr/public/11430252,서울시본청,복지본부,장애인복지정책과,NaN,NaN,2017,1,NaN,NaN,NaN,복지본부 장애인복지정책과,2017-01-23 15:10,서울시청신매점,부서운영에 필요한 음료수 구매,장애인복지정책과직원,카드,16000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5512,15513286,2017년 1월 서울시본청_경제진흥본부_경제기획관_투자유치과 업무추진비 내역,http://opengov.seoul.go.kr/public/15513286,서울시본청,경제진흥본부,경제기획관,투자유치과,NaN,2017,1,NaN,NaN,NaN,경제진흥본부 투자유치과,2017-01-11 13:01,차이79 종로구 새문안로5길,금융기관 보조금 지원계획수립관련 간담회 개최 비용지출,투자유치과장외5명,카드,27000
5513,15513286,2017년 1월 서울시본청_경제진흥본부_경제기획관_투자유치과 업무추진비 내역,http://opengov.seoul.go.kr/public/15513286,서울시본청,경제진흥본부,경제기획관,투자유치과,NaN,2017,1,NaN,NaN,NaN,경제진흥본부 투자유치과,2017-01-06 12:58,강가 서울 중구 세종대로136,핀테크 산업지원시설 입주 관련 간담회 개최비용지출,금융협력관외6명,카드,70000
5514,15513286,2017년 1월 서울시본청_경제진흥본부_경제기획관_투자유치과 업무추진비 내역,http://opengov.seoul.go.kr/public/15513286,서울시본청,경제진흥본부,경제기획관,투자유치과,NaN,2017,1,NaN,NaN,NaN,경제진흥본부 투자유치과,2017-01-04 12:47,이복만두 중구 무교동 27,F사 투자유치 협의를 위한 간담회 개최 비용지출,투자유치과장외4명,카드,57000
5515,15513286,2017년 1월 서울시본청_경제진흥본부_경제기획관_투자유치과 업무추진비 내역,http://opengov.seoul.go.kr/public/15513286,서울시본청,경제진흥본부,경제기획관,투자유치과,NaN,2017,1,NaN,NaN,NaN,경제진흥본부 투자유치과,2017-01-03 12:42,불고기브라더스 중구 태평로 1가,핀테크사업 전략수립을 위한 간담회 개최비용지출,경제진흥본부장외10명,카드,152400
